<a href="https://colab.research.google.com/github/rajaih/Perci-rajai/blob/master/Rajai_Linear_Regession.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import openai
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import json
from pandas import json_normalize

# Load the JSON file
with open('/content/MC_TITAN_2024-01-02_2024-12-31_1.json') as f:
    data1 = json.load(f)

# Instead of treating 't' as record_path, directly create a DataFrame
# and rename the column containing timestamps to 't'
df = pd.DataFrame(data1['t'], columns=['t'])

# Include 's' as a separate column with the same value for all rows
df['open'] = data1['o']
df['high'] = data1['h']
df['low'] = data1['l']
df['close'] = data1['c']
df['volume'] = data1['v']
# Convert 't' column to datetime objects
df['t'] = pd.to_datetime(df['t'], unit='s', utc=True) # Assuming 't' is in Unix timestamps (seconds)

# Set timezone to Asia/Kolkata
df['t'] = df['t'].dt.tz_convert('Asia/Kolkata')

# If you need a separate df2 DataFrame, you can create it now
df2 = df.copy()

In [2]:
df2.head(5)

,t,open,high,low,close,volume
0,2024-04-18 09:15:00+05:30,3643.70,3643.70,3626.35,3640.25,15977
1,2024-04-18 09:16:00+05:30,3641.85,3648.00,3641.85,3646.55,9481
2,2024-04-18 09:17:00+05:30,3646.75,3646.75,3641.00,3646.45,7980
3,2024-04-18 09:18:00+05:30,3645.20,3650.00,3645.00,3646.95,7341
4,2024-04-18 09:19:00+05:30,3646.95,3649.05,3644.10,3645.25,3696


In [3]:
df2['Target'] = df2['close'].shift(-1)
df2.dropna(inplace=True)

In [26]:
X11 = df2[['open', 'close']]
y11 = df2['Target']


In [27]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt

In [28]:
X_trainl, X_testl, y_trainl, y_testl = train_test_split(X11, y11, test_size=0.2, shuffle=False)

In [29]:
modell = LinearRegression()
modell.fit(X_trainl, y_trainl)

LinearRegression()

In [30]:
y_predl = modell.predict(X_testl)

print("Mean Squared Error:", mean_squared_error(y_testl, y_predl))
print("R² Score:", r2_score(y_testl, y_predl))

Mean Squared Error: 5.605023124492406
R² Score: 0.999452029049894


In [9]:
df3 = df2.copy()

In [10]:
#labeling sentiments
df['Price_Change'] = df['close'].diff()
df['Sentiment'] = df['Price_Change'].apply(lambda x: 1 if x > 0 else (-1 if x < 0 else 0))
df['sentimentScore'] = df['close'].apply(lambda x: 1 if x > 3640 else 0)


In [11]:
df.head(5)

,t,open,high,low,close,volume,Price_Change,Sentiment,sentimentScore
0,2024-04-18 09:15:00+05:30,3643.70,3643.70,3626.35,3640.25,15977,NaN,0,1
1,2024-04-18 09:16:00+05:30,3641.85,3648.00,3641.85,3646.55,9481,6.3,1,1
2,2024-04-18 09:17:00+05:30,3646.75,3646.75,3641.00,3646.45,7980,-0.1,-1,1
3,2024-04-18 09:18:00+05:30,3645.20,3650.00,3645.00,3646.95,7341,0.5,1,1
4,2024-04-18 09:19:00+05:30,3646.95,3649.05,3644.10,3645.25,3696,-1.7,-1,1


In [12]:
df['buysell'] = df.apply(lambda x: "Buy" if x['close'] > x['open'] else "sell", axis=1)

In [13]:
X= df[['open','close']]
y= df['sentimentScore']
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [14]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier()
model.fit(X_train, y_train)

RandomForestClassifier()

In [15]:
y_pred = model.predict(X_test)

In [ ]:
print(y_pred)

[1 0 1 ... 0 1 0]


In [16]:
X1= df[['open','close']]
y1= df['buysell']
from sklearn.model_selection import train_test_split

X_train1, X_test1, y_train1, y_test1 = train_test_split(X1, y1, test_size=0.2, random_state=42)
#
model1 = RandomForestClassifier()
model1.fit(X_train1, y_train1)

RandomForestClassifier()

In [ ]:
y_pred1 = model.predict(X1)
print(y_pred1)

[1 1 1 ... 0 0 0]


In [ ]:
df['predicted_sentiment'] = y_pred1

In [ ]:
df.head(5)

,t,open,high,low,close,volume,Price_Change,Sentiment,sentimentScore,buysell,predicted_sentiment
0,2024-04-18 09:15:00+05:30,3643.70,3643.70,3626.35,3640.25,15977,NaN,0,1,sell,1
1,2024-04-18 09:16:00+05:30,3641.85,3648.00,3641.85,3646.55,9481,6.3,1,1,Buy,1
2,2024-04-18 09:17:00+05:30,3646.75,3646.75,3641.00,3646.45,7980,-0.1,-1,1,sell,1
3,2024-04-18 09:18:00+05:30,3645.20,3650.00,3645.00,3646.95,7341,0.5,1,1,Buy,1
4,2024-04-18 09:19:00+05:30,3646.95,3649.05,3644.10,3645.25,3696,-1.7,-1,1,sell,1


In [ ]:
df3.head()

,t,open,high,low,close,volume,Target
0,2024-04-18 09:15:00+05:30,3643.70,3643.70,3626.35,3640.25,15977,3646.55
1,2024-04-18 09:16:00+05:30,3641.85,3648.00,3641.85,3646.55,9481,3646.45
2,2024-04-18 09:17:00+05:30,3646.75,3646.75,3641.00,3646.45,7980,3646.95
3,2024-04-18 09:18:00+05:30,3645.20,3650.00,3645.00,3646.95,7341,3645.25
4,2024-04-18 09:19:00+05:30,3646.95,3649.05,3644.10,3645.25,3696,3643.80


In [33]:
y_predl2 = modell.predict(X11)

In [35]:
df3['PredcitedTarget'] = y_predl2

In [36]:
df3.head(5)

,t,open,high,low,close,volume,Target,PredcitedTarget
0,2024-04-18 09:15:00+05:30,3643.70,3643.70,3626.35,3640.25,15977,3646.55,3640.221883
1,2024-04-18 09:16:00+05:30,3641.85,3648.00,3641.85,3646.55,9481,3646.45,3646.515430
2,2024-04-18 09:17:00+05:30,3646.75,3646.75,3641.00,3646.45,7980,3646.95,3646.418978
3,2024-04-18 09:18:00+05:30,3645.20,3650.00,3645.00,3646.95,7341,3645.25,3646.917473
4,2024-04-18 09:19:00+05:30,3646.95,3649.05,3644.10,3645.25,3696,3643.80,3645.220099
